In [1]:
from sklearn.datasets import fetch_20newsgroups

# Uncomment this to learn more about the API.
# help(fetch_20newsgroups)

We load the 20 Newsgroups data, keeping only 4 categories.

In [2]:
categories = ["alt.atheism", "soc.religion.christian", "comp.graphics", "sci.med"]

# We use a fixed random state to ensure the shuffle is deterministic.
twenty_train = fetch_20newsgroups(
    subset="train",
    categories=categories,
    shuffle=True,
    random_state=42,
)
twenty_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
)

We create a basic text processing pipeline - bag of words features and Logistic Regression as a classifier.

See [How Count Vectorizer works](./refresher/001_how_count_vectorizer_works.md)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()

# We increase the number of iterations to prevent the error below:
# ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT
clf = LogisticRegressionCV(max_iter=1_000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('logisticregressioncv', LogisticRegressionCV(max_iter=1000))])

To check the quality of the model, we run it against the test data.

In [4]:
from sklearn import metrics


def print_report(pipe):
    y_test = twenty_test.target
    y_pred = pipe.predict(twenty_test.data)
    report = metrics.classification_report(
        y_test, y_pred, target_names=twenty_test.target_names
    )
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [5]:
print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.91      0.81      0.86       319
         comp.graphics       0.86      0.94      0.90       389
               sci.med       0.92      0.81      0.86       396
soc.religion.christian       0.88      0.98      0.93       398

              accuracy                           0.89      1502
             macro avg       0.89      0.89      0.89      1502
          weighted avg       0.89      0.89      0.89      1502

accuracy: 0.889


The model achieve a relatively high accuracy. To understand what the model learned, we use the `eli5.show_weights` function. The function accepts the classifier as the first argument:

In [6]:
import eli5

eli5.show_weights(clf, vec=vec, top=10, target_names=twenty_test.target_names)

How to interpret:

- columns are target calsses
- In each column, there are features and their weights.
- Intercept (bias) feature is shown as `<BIAS>` in the same table.


Documents with words `graphics`, `images` are closely related to the class `comp.graphics`.

In [7]:
eli5.show_prediction(
    clf, twenty_test.data[0], vec=vec, target_names=twenty_test.target_names
)

We can visualize how the predictions are made, and the highlighted feature (words) that contributes to the prediction.

> NOTE: Hovering on the highlighted words shows you the weight of the word.


We notice that the classifier learned some non-interesting data, such as email addresses. To improve the model performance, we should perform cleaning. 

We can take that step by first removing the `headers` and `footers`.

In [8]:
twenty_train = fetch_20newsgroups(
    subset="train",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=["headers", "footers"],
)

twenty_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=["headers", "footers"],
)

vec = CountVectorizer()
clf = LogisticRegressionCV(max_iter=1_000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target);

In [9]:
print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.81      0.76      0.79       319
         comp.graphics       0.82      0.93      0.87       389
               sci.med       0.87      0.78      0.82       396
soc.religion.christian       0.86      0.88      0.87       398

              accuracy                           0.84      1502
             macro avg       0.84      0.84      0.84      1502
          weighted avg       0.84      0.84      0.84      1502

accuracy: 0.840


The accuracy of the model somehow becomes worse (from `0.889` to `0.840`).

In [10]:
eli5.show_prediction(
    clf, twenty_test.data[0], vec=vec, target_names=twenty_test.target_names
)

Email addresses no longer appears in the result, but the classifier still doesn't look good. The classifier seems to assign high weights to non-related words such as `my`, `to`.

## Pipeline Improvement

To improve the classifier, we may filter out stop words.

In [11]:
vec = CountVectorizer(stop_words="english")
clf = LogisticRegressionCV(max_iter=1_000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.86      0.76      0.81       319
         comp.graphics       0.85      0.94      0.89       389
               sci.med       0.92      0.85      0.88       396
soc.religion.christian       0.86      0.89      0.87       398

              accuracy                           0.87      1502
             macro avg       0.87      0.86      0.86      1502
          weighted avg       0.87      0.87      0.87      1502

accuracy: 0.868


In [12]:
eli5.show_prediction(
    clf, twenty_test.data[0], vec=vec, target_names=twenty_test.target_names
)

The accuracy of the model is now `0.868`, which is better than the previous (`0.840`), but still not as good as the first (`0.889`). 

But the highlighted words seems to be more relevant.

We can also experiment with using TFIDF vectorizer.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(stop_words="english")
clf = LogisticRegressionCV(max_iter=1_000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.92      0.80      0.86       319
         comp.graphics       0.90      0.96      0.93       389
               sci.med       0.95      0.92      0.93       396
soc.religion.christian       0.89      0.94      0.91       398

              accuracy                           0.91      1502
             macro avg       0.91      0.91      0.91      1502
          weighted avg       0.91      0.91      0.91      1502

accuracy: 0.911


In [14]:
eli5.show_prediction(
    clf,
    twenty_test.data[0],
    vec=vec,
    target_names=twenty_test.target_names,
    targets=["sci.med"],
)

## Debugging HashingVectorizer

Instead of char n-grams, we try fitting word n-grams. 

To handle large vocabularies, we can use HashingVectorizer, to make training faster we can employ SGDClassifier:

In [15]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vec = HashingVectorizer(stop_words="english", ngram_range=(1, 2))
clf = SGDClassifier(max_iter=20, random_state=42)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.92      0.79      0.85       319
         comp.graphics       0.87      0.96      0.92       389
               sci.med       0.93      0.89      0.91       396
soc.religion.christian       0.88      0.93      0.91       398

              accuracy                           0.90      1502
             macro avg       0.90      0.89      0.90      1502
          weighted avg       0.90      0.90      0.90      1502

accuracy: 0.899


The classifier ran super fast. Let's check what the model learned.

In [16]:
eli5.show_prediction(
    clf,
    twenty_test.data[0],
    vec=vec,
    target_names=twenty_test.target_names,
    targets=["sci.med"],
)

The result looks similar to `CountVectorizer`. But with `HashingVectorizer` we don't even have a vocabulary. How does this work?

In [17]:
eli5.show_weights(clf, vec=vec, top=10, target_names=twenty_test.target_names)

The feature is not what we expect - since we don't have vocabulary, we don't have feature names. We can use `InvertableHashingVectorizer` to get feature names for `HashingVectorizer` without fitting a huge vocabulary.

We still need some data to learn the words to hash mapping, and we can use a random subset of data to fit it.

In [18]:
import numpy as np
from eli5.sklearn import InvertableHashingVectorizer

In [19]:
ivec = InvertableHashingVectorizer(vec)
sample_size = len(twenty_train.data) // 10
X_sample = np.random.choice(twenty_train.data, size=sample_size)
ivec.fit(X_sample)

InvertableHashingVectorizer(vec=HashingVectorizer(ngram_range=(1, 2),
                                                  stop_words='english'))

In [20]:
eli5.show_weights(clf, vec=ivec, top=20, target_names=twenty_test.target_names)

There are collisions (hover mouse over features with "..."), and there important features which were not seen in the random sample (FEATURE[...]), but overall it looks fine.

The `rutgets edu` bigram is suspicious though, it looks like a part of email.

In [21]:
rutgers_example = [x for x in twenty_train.data if "rutgers" in x.lower()][0]
rutgers_example

"In article <Apr.8.00.57.41.1993.28246@athos.rutgers.edu> REXLEX@fnal.gov writes:\n>In article <Apr.7.01.56.56.1993.22824@athos.rutgers.edu> shrum@hpfcso.fc.hp.com\n>Matt. 22:9-14 'Go therefore to the main highways, and as many as you find\n>there, invite to the wedding feast.'...\n\n>hmmmmmm.  Sounds like your theology and Christ's are at odds. Which one am I \n>to believe?"

Yup, it seems like the model learned this email address instead of something useful.

In [22]:
eli5.show_prediction(
    clf,
    rutgers_example,
    vec=vec,
    target_names=twenty_test.target_names,
    targets=["soc.religion.christian"],
)